In [1]:
test_text = "CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Your effort and contribution in providing this feedback is much Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Your effort and contribution in providing this feedback is much Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Your effort and contribution in providing this feedback is much CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Your effort and contribution in providing this feedback is much CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously CNN values your feedback CNN values your feedback 1. How relevant is this ad to you? 1. How relevant is this ad to you? 2. Did you encounter any technical issues? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Your effort and contribution in providing this feedback is much Your effort and contribution in providing this feedback is much Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Topics You Follow Topics You Follow Topics You Follow Your CNN account Sign in to your CNN account Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Topics You Follow Topics You Follow Topics You Follow Your CNN account Sign in to your CNN account Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Fear & Greed Fear & Greed Fear & Greed Fear & Greed Life, But Better Life, But Better Life, But Better Life, But Better Food & Drink Food & Drink Food & Drink Food & Drink Call to Earth Impact Your World Call to Earth Impact Your World Call to Earth Call to Earth Impact Your World Impact Your World CNN TV Schedules CNN TV Schedules CNN TV Schedules CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts CNN 5 Things CNN 5 Things Chasing Life with Dr. Sanjay Gupta Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish The Assignment with Audie Cornish Tug of War Tug of War CNN Political Briefing CNN Political Briefing The Axe Files The Axe Files All There Is with Anderson Cooper All There Is with Anderson Cooper All CNN Audio podcasts All CNN Audio podcasts Work for CNN Work for CNN Work for CNN Work for CNN At Syria’s border with Lebanon, refugees return home while others flee fearing the worst 6 minute read 6:08 AM EST, Thu December 12, 2024 Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. At Syria’s border with Lebanon, refugees return home while others flee fearing the worst 6 minute read 6:08 AM EST, Thu December 12, 2024 At Syria’s border with Lebanon, refugees return home while others flee fearing the worst 6 minute read 6:08 AM EST, Thu December 12, 2024 At Syria’s border with Lebanon, refugees return home while others flee fearing the worst At Syria’s border with Lebanon, refugees return home while others flee fearing the worst 6 minute read 6:08 AM EST, Thu December 12, 2024 6 minute read 6:08 AM EST, Thu December 12, 2024 6 minute read 6:08 AM EST, Thu December 12, 2024 6 minute read 6:08 AM EST, Thu December 12, 2024 6 minute read 6:08 AM EST, Thu December 12, 2024 Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Nadia’s son fled the war in Syria nine years ago. Following the collapse of the Assad regime, she is finally able to visit him. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Masnaa, Lebanon’s border with Syria Masnaa, Lebanon’s border with Syria Masnaa, Lebanon’s border with Syria Nadia’s eyes were full of tears as she crossed the border from Syria to Lebanon. She was finally going to see her son. A 14-year-old boy the last time she saw him; he is now 22 and living in Germany. Her son left the family home in Hasaka in northern Syria in 2013 during one of the many escalations of the war which had at that point been raging for more than two years. Nadia, her husband Saiman and their daughter Sydra stayed in Syria, never imagining it would be so long before they’d see Mohammed again. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Bashar al-Assad, seen during the Signing of the comprehensive program of strategic and long-term cooperation between Iran and Syria, in Damascus, Syria on May 03, 2023. Matin Ghasemi/Borna News/Aksonline ATPImages/Getty Images After decades of brutal rule, Bashar al-Assad’s regime has been toppled After decades of brutal rule, Bashar al-Assad’s regime has been toppled With a couple of suitcases in tow, Nadia and Saiman finally made it to Masnaa on the Lebanese side of the border on Wednesday afternoon. They were headed to Beirut to board a flight to Germany. fall of Syria’s longtime dictator Bashar al-Assad over the weekend triggered a wave of jubilant celebrations, with many Syrians rushing home after years in exile. But the astonishing speed with which the rebels took over the country and the subsequent Israeli attacks against Syrian government targets also sparked worries about what’s next for the battered country. fall of Syria’s longtime dictator Bashar al-Assad At the crossing in Masnaa, hundreds of people were streaming in both directions, a sense of nervousness and chaos in the air. Beeping vehicles were trying to weave their way through the traffic jam of cars and vans loaded to the roof with people and their possessions. Taxi drivers flocked to the crossing, offering rides to those on foot – some asking for extortionate amounts of money for a transfer to Beirut or Damascus. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. People cross from Syria into Lebanon at the Masnaa border crossing on Wednesday, December 11. Another couple, Mohammed and his wife, were among those returning to Syria. Waiting for their documents to be stamped at the Lebanese side of the border, they were all smiles. They had first met in Lebanon after both fled the war in Syria. Now, after 11 years abroad, they were taking their family home. “My sons were all born here, in Lebanon, this is their first time going to Syria,” Mohammed said, pointing at four little boys sitting next to each other inside the busy border building. Their feet dangling from the bench, the children looked excited about the prospect of returning to a home they had never known. “It’s been very long time, and I just want to go home, back to my village. My brother is there, I have not seen him in 11 years,” Mohammed told CNN. But returning home proved far more difficult than he thought. Officials told him that while he and his wife were fine to go, his children didn’t have the correct paperwork to cross the border – because they were born in Lebanon, not Syria. Suddenly, the smiles were all gone. As the family walked out of the building, a stranger came up offering to take them across the border on foot – for a hefty fee. Mohammed turned him away. Paperwork headaches have been a problem for many Syrians wanting to come home. Many had left the country under precarious conditions, often illegally. ‘I just want to keep my children safe’ Diaa and his family were heading the opposite way – from Syria back to Lebanon. He is from Lebanon, while his wife Amani is from Syria. The couple and their three children lived in Beirut, but fled to Damascus in September, when Israel launched an intense bombing campaign against Hezbollah in Lebanon. “I decided to send my family to Syria after Hassan Nasrallah was assassinated . The children were playing outside and I found them in a horrific state, they were completely consumed by fear,” he said. Hassan Nasrallah was assassinated Nasrallah, the long-time leader of the Iran-backed Hezbollah, was killed when Israeli jets hit the militant group’s headquarters in southern Beirut. The huge strike and subsequent explosions rocked the city, including the neighborhood where Diaa’s family lived. “My eldest daughter could not speak for an hour. It really terrified me. The children have not yet recovered from the (2020) Beirut port explosion; they are afraid of everything, they are very afraid even of the wind slamming the door,” he said. On Wednesday, Diaa and Amani found themselves once again running away from strikes. They were hauling a huge pile of bags from the border to a car parked on the Lebanese side, all five family members carrying what they could. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Amani, her husband Diaa and their children decided to return to Beirut on Wednesday. Diaa said they were moving because he was worried about Israel's strikes against Syria. Diaa said they were moving because he was worried about Israel's strikes against Syria. Diaa said they were moving because he was worried about Israel's strikes against Syria. Diaa said they were moving because he was worried about Israel's strikes against Syria. Israel has said it nearly 500 times in the past two days, destroying the country’s weapons stockpiles and naval fleet to prevent them falling “into the hands of extremists.” Meanwhile, Turkish state media reported a Turkish drone destroyed military equipment in northern Syria that had been seized by a Kurdish group. “The thing that struck me the most was when my little son, who is seven years old, told me on the phone that he wanted to move to Beirut three days ago. The idea of displacement is now deeply rooted in his mind,” he said. “Lebanon, war. Syria, war,” Diaa said, pointing across the border and back. “I just want to keep my children safe,” he said. While most people in Syria are celebrating the collapse of the Assad regime, many are worried about the future. Islamist rebel coalition that has now taken over Syria brings together a variety of armed rebel groups, each with its own ideology and ideas about what should happen next. That leaves Syria’s minority groups like Alawites, Shia Muslims, Ismailis, Druze, Kurds and Christians in fear of being targeted. Islamist rebel coalition Concerns over the precarious security situation and the potential for retribution has led some supporters of the Assad regime to flee the country. Many of the ousted regime’s senior figures come from the Alawite sect, a branch of Shia Islam. Just beyond the Masnaa border crossing, in the no man’s land between the two countries, thousands of people were waiting on Thursday to get permits to enter Lebanon. Many had been there for days, sleeping on the ground in the open air. “It gets so cold at night here. And there are so many children,” Fatima, a young woman trying to get into Lebanon, told CNN. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. In a strip of rugged terrain on the Syria-Lebanon border between the two checkpoints, thousands of people were waiting on Thursday, December 12, to get permits to enter Lebanon. She said she and her family fled Damascus after armed rebels entered their home three days ago, ordering them to leave. “We are Shia and they said we have to go or they’ll kill us. All Shia will have to leave Syria now,” she said. Fatima said she and her family and friends were celebrating the fall of Assad. “We hate him. He’s a criminal, he killed a lot of people. We hate him, we hate him. But you know, there’s a lot of people who hate us too.” The Lebanese General Security intelligence agency said that some people without permits were trying to enter Lebanon by force at the Masnaa crossing and that the Lebanese army and security agencies had now been deployed there to take control of the situation. DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) DAMASCUS, SYRIA - DECEMBER 8: People wave guns in the air, as they gather to celebrate the fall of the Syrian regime in Umayyad Square on December 8, 2024 in Damascus, Syria. Rebel forces in Syria claimed that they had retaken the capital from longtime ruler Bashar al-Assad, who was reported to have fled the country. Syria has been mired in a multi-party civil war since 2011, amid the Arab Spring uprisings. (Photo by Ali Haj Suleiman/Getty Images) Ali Haj Suleiman/Getty Images Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Syria is free of its dictator. The rebels’ biggest challenge now is learning how to govern Saiman and Nadia, the couple traveling to Germany to meet their son, said the uncertainty and fear of more violence were among the reasons why they had decided to leave now. As Kurds from northern Syria, they are worried about another escalation in the long-running conflict between Kurdish insurgents and Turkey-backed militant groups. “Even the areas where there are no problems, we are always worried there would be problems. There are all sorts of (armed men who) came from all over the world to Syria to join al Qaeda. As Kurds, we are especially afraid,” Saiman told CNN. The journey across the border was a bittersweet moment for the couple. Weeping, Nadia explained that they were forced to leave behind their daughter Sydra, who is now 18. She has not yet received her travel permits and is staying with the neighbors for the time being. As Nadia and Saiman continued their journey to Beirut, Mohammed, his wife and their four sons headed the other way. They made it to Syria on Wednesday evening. “Thank God, we have arrived,” Mohammed told CNN in a text message. “They welcomed us and my children with great hospitality and honor. I did not find my house though. It was destroyed by a plane (strike),” he said. CNN’s Sarah El Sirgany contributed to this report. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network. Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Topics You Follow Topics You Follow Topics You Follow Your CNN account Sign in to your CNN account Your CNN account Sign in to your CNN account Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Fear & Greed Life, But Better Food & Drink Call to Earth Impact Your World CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts Work for CNN Fear & Greed Fear & Greed Fear & Greed Fear & Greed Life, But Better Life, But Better Life, But Better Life, But Better Food & Drink Food & Drink Food & Drink Food & Drink Call to Earth Impact Your World Call to Earth Impact Your World Call to Earth Call to Earth Impact Your World Impact Your World CNN TV Schedules CNN TV Schedules CNN TV Schedules CNN TV Schedules CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts CNN 5 Things Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish Tug of War CNN Political Briefing The Axe Files All There Is with Anderson Cooper All CNN Audio podcasts CNN 5 Things CNN 5 Things Chasing Life with Dr. Sanjay Gupta Chasing Life with Dr. Sanjay Gupta The Assignment with Audie Cornish The Assignment with Audie Cornish Tug of War Tug of War CNN Political Briefing CNN Political Briefing The Axe Files The Axe Files All There Is with Anderson Cooper All There Is with Anderson Cooper All CNN Audio podcasts All CNN Audio podcasts Work for CNN Work for CNN Work for CNN Work for CNN Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Your CNN account Sign in to your CNN account Topics You Follow Topics You Follow Topics You Follow Topics You Follow Your CNN account Sign in to your CNN account Your CNN account Sign in to your CNN account Terms of Use Accessibility & CC Terms of Use Accessibility & CC © 2024 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved. CNN Sans ™ & © 2016 Cable News Network."

In [72]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator= " ",
    chunk_size=2000,  # Slightly larger chunk size
    chunk_overlap=200,  # Significant overlap to maintain context
)

In [73]:
split_docs = text_splitter.split_text(test_text)

In [74]:
len(split_docs)

29

In [19]:
CharacterTextSplitter??

Init signature:
CharacterTextSplitter(
    separator: 'str' = '\n\n',
    is_separator_regex: 'bool' = False,
    **kwargs: 'Any',
) -> 'None'
Source:        
class CharacterTextSplitter(TextSplitter):
    """Splitting text that looks at characters."""

    def __init__(
        self, separator: str = "\n\n", is_separator_regex: bool = False, **kwargs: Any
    ) -> None:
        """Create a new TextSplitter."""
        super().__init__(**kwargs)
        self._separator = separator
        self._is_separator_regex = is_separator_regex

    def split_text(self, text: str) -> List[str]:
        """Split incoming text and return chunks."""
        # First we naively split the large input into a bunch of smaller ones.
        separator = (
            self._separator if self._is_separator_regex else re.escape(self._separator)
        )
        splits = _split_text_with_regex(text, separator, self._keep_separator)
        _separator = "" if self._keep_separator else self._separator
        r

In [19]:
from html.parser import HTMLParser
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.hyperlinks = []

    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        if tag == "a" and "href" in attrs:   
            self.hyperlinks.append(attrs["href"])

In [20]:
from urllib.parse import urlparse
import urllib.request

def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

In [21]:
HTTP_URL_PATTERN = r'^http[s]*://.+'
import re

def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            # if url_obj.netloc == local_domain:
            #     clean_link = link
            clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return clean_links

In [22]:
domains = get_domain_hyperlinks('lobste','https://lobste.rs')

In [24]:
 domains

['https://lobste/~felixyz',
 'https://lobste/s/f9a6vj/announcing_netbsd_10_1_dec_16_2024',
 'https://ghostarchive.org/search?term=https%3A%2F%2Fgithub.com%2Fergochat%2Fergo',
 'https://web.archive.org/web/3/https%3A%2F%2Fchapel-lang.org%2Fblog%2Fposts%2Fannouncing-chapel-2.3%2F',
 'https://web.archive.org/web/3/https%3A%2F%2Fkorban.net%2Fposts%2Felm%2F2024-11-16-typescript-react-impressions%2F',
 'https://lobste/t/performance',
 'https://ghostarchive.org/search?term=https%3A%2F%2Fslint.dev%2Fblog%2Fslint-1.9-released',
 'https://wiki.alopex.li/SurveyOfSystemLanguages2024',
 'https://web.archive.org/web/3/https%3A%2F%2Fpurplesyringa.moe%2Fblog%2Fthe-ram-myth%2F',
 'https://lobste/s/blg0vs/ram_myth',
 'https://lobste/tags',
 'https://lobste/domains/youtu.be',
 'https://lobste/t/perl',
 'https://lobste/filters',
 'https://lobste/t/elm',
 'https://lobste/~jencelpanic',
 'https://archive.today/https%3A%2F%2Fwww.netbsd.org%2Freleases%2Fformal-10%2FNetBSD-10.1.html',
 'https://byroot.github.i

In [1]:
!pip3 install newspaper3k

  Using cached newspaper3k-0.2.8-py3-none-any.whl.metadata (11 kB)
  Using cached cssselect-1.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached tldextract-5.1.3-py3-none-any.whl.metadata (11 kB)
  Using cached feedfinder2-0.0.4-py3-none-any.whl
  Using cached jieba3k-0.35.1-py3-none-any.whl
  Using cached tinysegmenter-0.3-py3-none-any.whl
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
  Using cached requests_file-2.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached newspaper3k-0.2.8-py3-none-any.whl (211 kB)
Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached tldextract-5.1.3-py3-none-any.whl (104 kB)
Using cached requests_file-2.1.0-py2.py3-none-any.whl (4.2 kB)


In [ ]:
url = 'https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090'

In [34]:
from bs4 import BeautifulSoup
import requests


In [35]:
def extract_domain_name(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Get the domain (hostname)
    hostname = parsed_url.hostname or parsed_url.path
    # Remove subdomains and top-level domains (TLDs)
    domain_parts = hostname.split('.')
    if len(domain_parts) > 2:
        # For domains with subdomains like news.ycombinator.com
        return domain_parts[-2]
    elif len(domain_parts) == 2:
        # For domains without subdomains like lobste.rs
        return domain_parts[0]
    else:
        # Return as is (rare case)
        return hostname

In [36]:
def remove_elements_with_few_words(arr):
    # Filter out elements where the number of words is less than 3
    return [element for element in arr if len(element.split()) >= 3]

In [37]:
def clean_body_content(body_content):
    print("Type of body content: ", type(body_content))
    soup = BeautifulSoup(body_content, "html.parser")

    for script_or_style in soup(["script", "style"]):
        script_or_style.extract()

    # Get text or further process the content
    cleaned_content = soup.get_text(separator="\n")
    # print(cleaned_content)
    cleaned_content = "\n".join(
        line.strip() for line in cleaned_content.splitlines() if line.strip()
    )

    cleaned_content_array = cleaned_content.split('\n')
    filtered_cleaned_content_array = remove_elements_with_few_words(cleaned_content_array)
    filtered_cleaned_content = " ".join(filtered_cleaned_content_array)

    return filtered_cleaned_content

In [38]:
def get_webtext(url):
    local_domain = extract_domain_name(url)
    print("Local domain: ", local_domain)

    #Hyperlinks in the URL
    url_hyperlink = get_domain_hyperlinks(local_domain=local_domain, url=url)
    print("URL Hyperlinks: ", url_hyperlink)

    # Get the text from the URL using BeautifulSoup
    soup = BeautifulSoup(requests.get(url).text, "html.parser")


    text = str(soup.body())
    cleaned_body_content = clean_body_content(text)
    #print("Cleaned body content: ", cleaned_body_content)
        
    #take_screenshot(url, local_domain)    
    return cleaned_body_content

In [40]:
text = get_webtext('https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090')

Local domain:  microsoft
URL Hyperlinks:  ['https://microsoft/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090/replies/4357488', 'https://huggingface.co/collections/microsoft/phi-3-6626e15e9585a200d2d761e3', 'https://support.microsoft.com/contactus', 'https://microsoft/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090/replies/4357361', 'https://blogs.windows.com/windowsexperience/2024/12/06/phi-silica-small-but-mighty-on-device-slm', 'https://microsoft/t5/s/gxcuf89792/auth/oidcss/sso_login_redirect/provider/default?referer=https%3A%2F%2Ftechcommunity.microsoft.com%2Fblog%2Faiplatformblog%2Fintroducing-phi-4-microsoft%25E2%2580%2599s-newest-small-language-model-specializing-in-comple%2F4357090', 'https://www.microsoft.com/en-us/microsoft-teams/group-chat-software', 'https://account.microsoft.com', 'https://www.microsoft.com/en-us/microsoft-cloud', 'https

In [41]:
text

"Skip to content Microsoft Community Hub Artificial Intelligence and Machine Learning AI - AI Platform Blog AI - AI Platform Blog 2 MIN READ Introducing Phi-4: Microsoft’s Newest Small Language Model Specializing in Complex Reasoning Dec 13, 2024 Learn about Phi-4, the latest small language model in Phi family, that offers high quality results at a small size (14B parameters). Today we are introducing , our 14B parameter state-of-the-art small language model (SLM) that excels at complex reasoning in areas such as math, in addition to conventional language processing. Phi-4 is the latest member of our Phi family of small language models and demonstrates what’s possible as we continue to probe the boundaries of SLMs. Phi-4 is currently available on Azure AI Foundry under a Microsoft Research License Agreement (MSRLA) and will be available on Phi-4 outperforms comparable and larger models on math related reasoning due to advancements throughout the processes, including the use of high-qua

In [1]:
!pip install goose3

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=0386f88f8a2a03633df1bad5b6107bdf222b4cec86d991e470101f3faa0734ab
  Stored in directory: /home/antash/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
url = 'https://techcommunity.microsoft.com/blog/aiplatformblog/introducing-phi-4-microsoft%E2%80%99s-newest-small-language-model-specializing-in-comple/4357090'

In [9]:
from goose3 import Goose

g = Goose()

In [10]:
article = g.extract(url = 'https://lobste.rs/')

In [11]:
article.title

'Lobsters'

In [12]:
article.cleaned_text

''

In [44]:
article.infos

{'meta': {'description': 'View CNN world news today for international news and videos from Europe, Asia, Africa, the Middle East and the Americas.',
  'lang': 'en',
  'keywords': 'world news, international news, world news today',
  'favicon': '/media/sites/cnn/apple-touch-icon.png',
  'canonical': 'https://www.cnn.com/world',
  'encoding': 'utf-8'},
 'image': None,
 'domain': 'www.cnn.com',
 'title': 'World',
 'cleaned_text': "1. How relevant is this ad to you?\n\nAd froze or did not finish loading Video content did not start after ad Audio on ad was too loud\n\nAd was repetitive to ads I've seen previously",
 'opengraph': {'title': 'World | CNN',
  'type': 'website',
  'url': 'https://www.cnn.com/world',
  'site_name': 'CNN'},
 'tags': [],
 'tweets': [],
 'movies': [],
 'links': [],
 'authors': [],
 'publish_date': None}

In [32]:
top_image = article.schema

In [34]:
top_image

In [35]:
article.infos

{'meta': {'description': "A paper from Anthropic's Alignment Science team on Alignment Faking in AI large language models",
  'lang': 'en',
  'keywords': '',
  'favicon': '/favicon.ico',
  'canonical': 'https://www.anthropic.com/research/alignment-faking/',
  'encoding': 'utf-8'},
 'image': None,
 'domain': 'www.anthropic.com',
 'title': 'Alignment faking in large language models',
 'cleaned_text': "Most of us have encountered situations where someone appears to share our views or values, but is in fact only pretending to do so—a behavior that we might call “alignment faking”. Alignment faking occurs in literature: Consider the character of Iago in Shakespeare’s Othello, who acts as if he’s the eponymous character’s loyal friend while subverting and undermining him. It occurs in real life: Consider a politician who claims to support a particular cause in order to get elected, only to drop it as soon as they’re in office.\n\nCould AI models also display alignment faking? When models are

In [1]:
text = "A collision between a passenger ferry and a navy speedboat near Mumbai, India, has resulted in at least 13 deaths, with 101 people rescued, as an investigation is underway into the incident that occurred while the ferry was en route to the Elephanta Caves tourist destination."

In [2]:
from groq import Groq
import os

client_groq = Groq(
  api_key=os.environ.get("GROQ_API_KEY")
)

def check_url_type(article_text):
    system_prompt = """
      Classify the given summary as either 'IsArticle' or 'NotArticle'. Base your classification on the presence of a clear topic, relevant details, organized structure, formal tone, and neutral perspective. Respond with one of the following, and only this exact classification:
        - IsArticle
        - NotArticle
      Wait for the user to provide the summary text.
    """
    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": article_text},
    ]
    completion = client_groq.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=message,
        temperature=1,
        top_p=1,
        max_tokens=1024,
        stream=False
    )
    result = completion.choices[0].message.content
    print("Response LLM: ", result)

    return result

In [3]:
check_url_type(text)

Response LLM:  IsArticle


'IsArticle'

In [6]:
from urllib.parse import urlparse
import urllib.request

def extract_domain_name(url):
    # Parse the URL
    parsed_url = urlparse(url)
    # Get the domain (hostname)
    hostname = parsed_url.hostname or parsed_url.path
    # Remove subdomains and top-level domains (TLDs)
    domain_parts = hostname.split('.')
    if len(domain_parts) > 2:
        # For domains with subdomains like news.ycombinator.com
        return domain_parts[-2]
    elif len(domain_parts) == 2:
        # For domains without subdomains like lobste.rs
        return domain_parts[0]
    else:
        # Return as is (rare case)
        return hostname

In [7]:
extract_domain_name('https://lobste.rs/')

'lobste'

In [13]:
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            # if url_obj.netloc == local_domain:
            #     clean_link = link
            clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return clean_links


In [ ]:
def get_summary(url, depth=1, max_depth=2, processed_urls=None):
    """
    Recursively scrape a URL, get content summaries, and navigate nested links up to a given depth.
    
    :param url: The starting URL to scrape.
    :param depth: Current depth of recursion.
    :param max_depth: Maximum recursion depth allowed.
    :param processed_urls: Set of URLs already processed to avoid redundancy.
    :return: A list of summaries for the current and nested URLs.
    """
    if depth > max_depth:
        # Stop recursion when max depth is reached
        return []

    if processed_urls is None:
        # Initialize the set to track processed URLs
        processed_urls = set()

    if url in processed_urls:
        # Skip already processed URLs
        print(f"Skipping already processed URL: {url}")
        return []

    print(f"Processing URL at depth {depth}: {url}")
    processed_urls.add(url)

    local_domain = extract_domain_name(url)
    print("Local-domain name:", local_domain)

    try:
        # Fetch and clean the content from the main URL
        cleaned_body_content = get_webtext(url)

        # Get the summary and page type
        llm_response, url_type = call_llm(cleaned_body_content)

        # Initialize the parent response
        parent_response = {"url": url, "page_type": url_type, "url_summary": llm_response}
        all_llm_response = [parent_response]

        # If the page type is valid, retrieve hyperlinks for nested scraping
        if url_type:
            url_hyperlinks = get_domain_hyperlinks(local_domain=local_domain, url=url)
            print(f"Total Number of hyperlinks found: {len(url_hyperlinks)}")
            unique_links = set(url_hyperlinks) - processed_urls  # Filter only unprocessed links

            for link in unique_links:
                try:
                    print(f"Processing nested link: {link}")

                    # Recursively fetch summaries for nested links
                    nested_summary = get_summary(link, depth=depth + 1, max_depth=max_depth, processed_urls=processed_urls)
                    if nested_summary:
                        all_llm_response.extend(nested_summary)

                except Exception as e:
                    print(f"Error processing link {link}: {e}")
                    continue  # Skip to the next link in case of an error

        return all_llm_response

    except Exception as e:def get_summary(url, depth=1, max_depth=2):
    if depth > max_depth:
        # Prevents excessive recursion
        return []

    local_domain = extract_domain_name(url)
    print("Local-domain name: ",  local_domain)

    try:
        # Fetch and clean the content from the main URL
        cleaned_body_content = get_webtext(url)
        
        # Get the summary and page type
        llm_response, url_type = call_llm(cleaned_body_content)
        
        # Initialize the parent response
        parent_response = {"url": url, "page_type": url_type, "url_summary": llm_response}
        all_llm_response = [parent_response]

        # If the page type is not null retrieve hyperlinks for nested scraping
        if url_type:
            url_hyperlinks = get_domain_hyperlinks(local_domain=local_domain, url=url)
            print("Total Number of hyperlinks: ", len(url_hyperlinks))
            for link in url_hyperlinks:
                try:
                    print("Processing Link: ", link)
                    
                    # Recursively fetch summaries for nested links
                    nested_summary = get_summary(link, depth=depth + 1, max_depth=max_depth)
                    print("Nested summary: ", nested_summary)

                    if nested_summary:
                        # Append the summary of a nested page if available
                        all_llm_response.extend(nested_summary)

                except Exception as def get_summary(url, depth=1, max_depth=2):
    if depth > max_depth:
        # Prevents excessive recursion
        return []

    local_domain = extract_domain_name(url)
    print("Local-domain name: ",  local_domain)

    try:
        # Fetch and clean the content from the main URL
        cleaned_body_content = get_webtext(url)
        
        # Get the summary and page type
        llm_response, url_type = call_llm(cleaned_body_content)
        
        # Initialize the parent response
        parent_response = {"url": url, "page_type": url_type, "url_summary": llm_response}
        all_llm_response = [parent_response]

        # If the page type is not null retrieve hyperlinks for nested scraping
        if url_type:
            url_hyperlinks = get_domain_hyperlinks(local_domain=local_domain, url=url)
            print("Total Number of hyperlinks: ", len(url_hyperlinks))
            for link in url_hyperlinks:
                try:
                    print("Processing Link: ", link)
                    
                    # Recursively fetch summaries for nested links
                    nested_summary = get_summary(link, depth=depth + 1, max_depth=max_depth)
                    print("Nested summary: ", nested_summary)

                    if nested_summary:
                        # Append the summary of a nested page if available
                        all_llm_response.extend(nested_summary)

                except Exception as e:
                    print(f"Error processing link {link}: {e}")
                    # Continue to the next link if an error occurs
                    continue

        return all_llm_response

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return []
e:
                    print(f"Error processing link {link}: {e}")
                    # Continue to the next link if an error occurs
                    continue

        return all_llm_response

    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return []

        print(f"Error processing URL {url}: {e}")
        return []

In [26]:
get_summary('https://edition.cnn.com/2024/12/18/asia/elephanta-caves-navy-boat-collision-mumbai-intl-latam/index.html')

Local-domain name:  cnn
Error processing URL https://edition.cnn.com/2024/12/18/asia/elephanta-caves-navy-boat-collision-mumbai-intl-latam/index.html: name 'get_webtext' is not defined


[]